# People per tax unit

Look at `XTOT` and UBI variables (`nu18`, `n1821`, `n21`, and calculated `n65`) in the PUF and CPS files.

## Setup

In [1]:
import pandas as pd
import numpy as np
import taxcalc as tc

## Load

In [2]:
UBI_COLS = ['nu18', 'n1820', 'n21']
N65_COLS = ['age_head', 'age_spouse', 'elderly_dependents']
USECOLS = ['XTOT', 'RECID'] + UBI_COLS + N65_COLS

In [3]:
puf = pd.read_csv('~/puf.csv', usecols=USECOLS)
cps = pd.read_csv(tc.Records.CODE_PATH + '/cps.csv.gz', usecols=USECOLS)

## Preprocess

In [4]:
def n65(age_head, age_spouse, elderly_dependents): 
    return ((age_head >= 65).astype(int) + 
            (age_spouse >= 65).astype(int) + 
            elderly_dependents) 

In [5]:
def add_age_features(df):
    df['persons'] = df[UBI_COLS].sum(axis=1)
    df['XTOT1'] = np.where(df.XTOT == 0, 1, df.XTOT)
    df['persons_minus_XTOT1'] = df.persons - df.XTOT1
    df['head65'] = df.age_head >= 65
    df['spouse65'] = df.age_spouse >= 65
    df['n65'] = n65(df.age_head, df.age_spouse, df.elderly_dependents)
    df['n2164'] = df.n21 - df.n65

In [6]:
add_age_features(puf)
add_age_features(cps)

## Analyze

### PUF

In [7]:
puf.groupby('XTOT').size()

XTOT
0     9583
1    86180
2    75820
3    30557
4    30015
5    16417
6       16
7        3
dtype: int64

In [8]:
puf.groupby('persons').size()

persons
1     91851
2     78208
3     31964
4     30084
5     16371
6        69
7        23
8        17
10        4
dtype: int64

#### Difference between `sum(nu18, n1820, n21)` and `max(XTOT, 1)`

Limit to records with `persons <= 5` due to `XTOT` top-coding at 5.

In [9]:
puf[puf.persons <= 5].groupby('persons_minus_XTOT1').size()

persons_minus_XTOT1
0    242959
1      5459
2        53
3         6
4         1
dtype: int64

Number of records with `persons <= 5` and `persons > XTOT1`.

In [10]:
puf[(puf.persons <= 5) & (puf.persons_minus_XTOT1 > 0)].shape[0]

5519

Percent of records with `persons <= 5` where `persons > XTOT1`.

In [11]:
(puf[(puf.persons <= 5) & (puf.persons_minus_XTOT1 > 0)].shape[0] /
 puf[(puf.persons <= 5)].shape[0])

0.022211221919043133

In [12]:
puf[puf.persons <= 5].sort_values('persons_minus_XTOT1', 
                                  ascending=False).head()

,elderly_dependents,age_head,age_spouse,n1820,n21,nu18,XTOT,RECID,persons,XTOT1,persons_minus_XTOT1,head65,spouse65,n65,n2164
247185,0,38,0,0,3,2,1,247186,5,1,4,False,False,0,3
245705,0,70,0,0,4,1,2,245706,5,2,3,True,False,1,3
244875,0,63,0,0,5,0,2,244876,5,2,3,False,False,0,5
244447,0,64,0,0,2,3,2,244448,5,2,3,False,False,0,2
248020,0,85,0,3,1,1,2,248021,5,2,3,True,False,1,0


In [13]:
puf[puf.n2164 < 0].shape

(715, 15)

In [14]:
pd.unique(puf.RECID.head())

array([1, 2, 3, 4, 5])

In [15]:
puf[puf.n2164 < 0].pivot_table(
    index=['n21'], 
    columns=['head65', 'spouse65', 'elderly_dependents'],
    values='RECID', aggfunc=len, fill_value=0)

head65              True          
spouse65           False True     
elderly_dependents     1     0   1
n21                               
1                      2   700   0
2                      0     0  13

Get example RECID values.

In [16]:
puf[(puf.n2164 < 0) & (puf.spouse65 == False)].RECID.head(1)

13274    13275
Name: RECID, dtype: int64

In [17]:
puf[(puf.n2164 < 0) & (puf.spouse65 == True) &
    (puf.elderly_dependents == 0)].RECID.head(1)

414    415
Name: RECID, dtype: int64

In [18]:
puf[(puf.n2164 < 0) & (puf.spouse65 == True) &
    (puf.elderly_dependents == 1)].RECID.head(1)

6868    6869
Name: RECID, dtype: int64

In [19]:
cps[cps.n2164 < 0].pivot_table(
    index=['n21'], 
    columns=['head65', 'spouse65', 'elderly_dependents'],
    values='RECID', aggfunc=len, fill_value=0)

head65,False,True
spouse65,False,False
elderly_dependents,1,1
n21,,
0,1,0
1,0,1


In [20]:
cps[cps.n2164 < 0]

,age_head,age_spouse,nu18,n1820,n21,elderly_dependents,XTOT,RECID,persons,XTOT1,persons_minus_XTOT1,head65,spouse65,n65,n2164
249110,85,0,0,0,1,1,1,249111,1,1,0,True,False,2,-1
379569,19,0,0,1,0,1,1,379570,1,1,0,False,False,1,-1
